In [1]:
# import libraries
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split

In [2]:
initialDf = pd.read_csv("steam.csv")
initialDf.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
for var in initialDf.columns:
    if len(initialDf[var]) != len(initialDf[var].dropna()):
        print(var)
print('Done')
#Check to see if there are any na's

Done


In [4]:
df = pd.DataFrame()
df["Name"] = initialDf["name"].apply(lambda x: str(x))
df["ReleaseYear"] = initialDf["release_date"].apply(lambda x: int(x[0:4]))
df["ReleaseMonth"] = initialDf["release_date"].apply(lambda x: int(x[5:7]))
df["English"] = initialDf["english"].apply(lambda x: int(x))
df["Developer"] = initialDf["developer"].apply(lambda x: str(x))
df["Publisher"] = initialDf["publisher"].apply(lambda x: str(x))
df["Windows"] = initialDf["platforms"].apply(lambda x: bool("windows" in x))
df["Mac"] = initialDf["platforms"].apply(lambda x: bool("mac" in x))
df["Linux"] = initialDf["platforms"].apply(lambda x: bool("linux" in x))
df["RequiredAge"] = initialDf["required_age"].apply(lambda x: int(x))
df["Categories"] = initialDf["categories"].apply(lambda x: str(x))
df["Genre"] = initialDf["genres"].apply(lambda x: str(x))
df["Price"] = initialDf["price"].apply(lambda x: float(x))


#df["PositiveRatings"] = initialDf["positive_ratings"].apply(lambda x: int(x))
#df["NegativeRatings"] = initialDf["negative_ratings"].apply(lambda x: int(x))
df["MedianPlaytime"] = initialDf["median_playtime"].apply(lambda x: int(x))
df["AveragePlaytime"] = initialDf["average_playtime"].apply(lambda x: int(x))
df["LowerOwners"] = initialDf["owners"].apply(lambda x: int(x[:x.find("-")]))
df["UpperOwners"] = initialDf["owners"].apply(lambda x: int(x[x.find("-")+1:]))

In [5]:
initialNormalizeFeatures = ['positive_ratings', 'negative_ratings']
NormalizeFeatures = ['PositiveRatings', 'NegativeRatings']
dfNormalize = pd.DataFrame(data=MinMaxScaler().fit_transform(initialDf[initialNormalizeFeatures]), columns=NormalizeFeatures)

df = pd.concat([df, dfNormalize], axis=1)
df.head()

D:\Program Files\Anaconda\envs\tf1.0\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Name,ReleaseYear,ReleaseMonth,English,Developer,Publisher,Windows,Mac,Linux,RequiredAge,Categories,Genre,Price,MedianPlaytime,AveragePlaytime,LowerOwners,UpperOwners,PositiveRatings,NegativeRatings
0,Counter-Strike,2000,11,1,Valve,Valve,True,True,True,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,7.19,317,17612,10000000,20000000,0.047093,0.006855
1,Team Fortress Classic,1999,4,1,Valve,Valve,True,True,True,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,3.99,62,277,5000000,10000000,0.001255,0.001300
2,Day of Defeat,2003,5,1,Valve,Valve,True,True,True,0,Multi-player;Valve Anti-Cheat enabled,Action,3.99,34,187,5000000,10000000,0.001292,0.000817
3,Deathmatch Classic,2001,6,1,Valve,Valve,True,True,True,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,3.99,184,258,5000000,10000000,0.000481,0.000548
4,Half-Life: Opposing Force,1999,11,1,Gearbox Software,Valve,True,True,True,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,3.99,415,624,5000000,10000000,0.001985,0.000591


In [6]:
df["NegativeRatings"].replace({0: 1}, inplace=True)

In [7]:
df['Pos_over_Neg'] =  (df['PositiveRatings'] / df['NegativeRatings']).apply(lambda x: float(x))

df['Ratio_over_std'] = np.where(df['Pos_over_Neg']>=df['Pos_over_Neg'].std(), 1, 0)

In [8]:
df.describe()

,ReleaseYear,ReleaseMonth,English,RequiredAge,Price,MedianPlaytime,AveragePlaytime,LowerOwners,UpperOwners,PositiveRatings,NegativeRatings,Pos_over_Neg,Ratio_over_std
count,27075.000000,27075.000000,27075.000000,27075.000000,27075.000000,27075.00000,27075.000000,2.707500e+04,2.707500e+04,27075.000000,27075.000000,27075.000000,27075.000000
mean,2016.498098,6.555457,0.981127,0.354903,6.078193,146.05603,149.804949,8.012669e+04,1.880543e+05,0.000378,0.107765,0.851269,0.149215
std,2.156990,3.459959,0.136081,2.406044,7.874922,2353.88008,1827.038141,8.708977e+05,1.786801e+06,0.007181,0.309515,1.476571,0.356307
min,1997.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000e+00,2.000000e+04,0.000000,0.000002,0.000000,0.000000
25%,2016.000000,3.000000,1.000000,0.000000,1.690000,0.00000,0.000000,0.000000e+00,2.000000e+04,0.000002,0.000008,0.162503,0.000000
50%,2017.000000,7.000000,1.000000,0.000000,3.990000,0.00000,0.000000,0.000000e+00,2.000000e+04,0.000009,0.000033,0.418658,0.000000
75%,2018.000000,10.000000,1.000000,0.000000,7.190000,0.00000,0.000000,2.000000e+04,5.000000e+04,0.000048,0.000220,0.959329,0.000000
max,2019.000000,12.000000,1.000000,18.000000,421.990000,190625.00000,190625.000000,1.000000e+08,2.000000e+08,1.000000,1.000000,47.981813,1.000000


In [9]:
df.head()

,Name,ReleaseYear,ReleaseMonth,English,Developer,Publisher,Windows,Mac,Linux,RequiredAge,...,Genre,Price,MedianPlaytime,AveragePlaytime,LowerOwners,UpperOwners,PositiveRatings,NegativeRatings,Pos_over_Neg,Ratio_over_std
0,Counter-Strike,2000,11,1,Valve,Valve,True,True,True,0,...,Action,7.19,317,17612,10000000,20000000,0.047093,0.006855,6.869742,1
1,Team Fortress Classic,1999,4,1,Valve,Valve,True,True,True,0,...,Action,3.99,62,277,5000000,10000000,0.001255,0.001300,0.965476,0
2,Day of Defeat,2003,5,1,Valve,Valve,True,True,True,0,...,Action,3.99,34,187,5000000,10000000,0.001292,0.000817,1.580898,1
3,Deathmatch Classic,2001,6,1,Valve,Valve,True,True,True,0,...,Action,3.99,184,258,5000000,10000000,0.000481,0.000548,0.878185,0
4,Half-Life: Opposing Force,1999,11,1,Gearbox Software,Valve,True,True,True,0,...,Action,3.99,415,624,5000000,10000000,0.001985,0.000591,3.357652,1


In [ ]:
def pos_neg(var):
    y = df['Pos_over_Neg']
    x = df[var]
    figure = plt.figure(figsize = (25,7))
    plt.scatter(x, y, marker='o');
    plt.ylabel('(Positive/Negative) Rating')
    plt.xlabel(var)

In [ ]:
pos_neg("Price")

In [ ]:
pos_neg("ReleaseYear")

In [ ]:
pos_neg("ReleaseMonth")

In [ ]:
pos_neg("AveragePlaytime")

In [ ]:
pos_neg("MedianPlaytime")

In [ ]:
plt.scatter(df.Windows, df["Pos_over_Neg"], label='Widnows')
plt.xlabel('On Windows? ')
plt.ylabel('(Positive/Negative) Rating')

In [ ]:
plt.scatter(df.Mac, df["Pos_over_Neg"], label='Mac')
plt.xlabel('On Mac?')
plt.ylabel('(Positive/Negative) Rating')

In [ ]:
plt.scatter(df.Linux, df["Pos_over_Neg"], label='Linux')
plt.xlabel('On Linux?')
plt.ylabel('(Positive/Negative) Rating')

In [ ]:
print(df.columns)

In [10]:
features = ['ReleaseYear', 'ReleaseMonth', 'English', 'Windows', 'Mac', 'Linux', 'RequiredAge', 'Price', 'MedianPlaytime', 'AveragePlaytime']
target = 'Ratio_over_std'

In [11]:
for col in df.select_dtypes(include=['bool']).columns:
    df[col] = df[col].apply(lambda x: int(1 if x else 0))
#for col in df.select_dtypes(include=['int']).columns:
#    df[col] = df[col].apply(lambda x: float(x))
df.head()

,Name,ReleaseYear,ReleaseMonth,English,Developer,Publisher,Windows,Mac,Linux,RequiredAge,...,Genre,Price,MedianPlaytime,AveragePlaytime,LowerOwners,UpperOwners,PositiveRatings,NegativeRatings,Pos_over_Neg,Ratio_over_std
0,Counter-Strike,2000,11,1,Valve,Valve,1,1,1,0,...,Action,7.19,317,17612,10000000,20000000,0.047093,0.006855,6.869742,1
1,Team Fortress Classic,1999,4,1,Valve,Valve,1,1,1,0,...,Action,3.99,62,277,5000000,10000000,0.001255,0.001300,0.965476,0
2,Day of Defeat,2003,5,1,Valve,Valve,1,1,1,0,...,Action,3.99,34,187,5000000,10000000,0.001292,0.000817,1.580898,1
3,Deathmatch Classic,2001,6,1,Valve,Valve,1,1,1,0,...,Action,3.99,184,258,5000000,10000000,0.000481,0.000548,0.878185,0
4,Half-Life: Opposing Force,1999,11,1,Gearbox Software,Valve,1,1,1,0,...,Action,3.99,415,624,5000000,10000000,0.001985,0.000591,3.357652,1


In [12]:
df.to_numpy()
print(df)

                             Name  ReleaseYear  ReleaseMonth  English  \
0                  Counter-Strike         2000            11        1   
1           Team Fortress Classic         1999             4        1   
2                   Day of Defeat         2003             5        1   
3              Deathmatch Classic         2001             6        1   
4       Half-Life: Opposing Force         1999            11        1   
...                           ...          ...           ...      ...   
27070             Room of Pandora         2019             4        1   
27071                   Cyber Gun         2019             4        1   
27072            Super Star Blast         2019             4        1   
27073  New Yankee 7: Deer Hunters         2019             4        1   
27074                   Rune Lord         2019             4        1   

                Developer             Publisher  Windows  Mac  Linux  \
0                   Valve                 Valve    

In [13]:
X = features
y = target

In [14]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten



Using TensorFlow backend.


In [15]:
from numpy import array
def split_s(sequences,n_steps):
    X,y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences)-1:
            break
        seq_x,seq_y = sequences.iloc[i:end_ix, :], sequences.iloc[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return(array(X),array(y))

In [64]:

X.shape[2]

1

In [80]:
n_steps = 10

X,y = split_s(df,n_steps)
X = np.expand_dims(X, axis=2)
X.shape[2]
n_features = X.shape[2]

ValueError: cannot copy sequence with size 10 to array axis with dimension 21

In [68]:
lenTrain, lenVal, lenTest = 25000, 1000, 1075
X_train, X_val, X_test = X[:lenTrain], X[lenTrain:(lenTrain+lenVal)], X[(lenTrain+lenVal):]
y_train, y_val, y_test = y[:lenTrain], y[lenTrain:(lenTrain+lenVal)], y[(lenTrain+lenVal):]
print(X_train)

[[['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]

 [['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]

 [['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]

 ...

 [['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]

 [['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]

 [['Name']
  ['ReleaseYear']
  ['ReleaseMonth']
  ...
  ['NegativeRatings']
  ['Pos_over_Neg']
  ['Ratio_over_std']]]


In [ ]:
'''
model.add(Flatten(input_shape=(1, 2)))
model = Sequential([
    Dense(32, input_dim=72),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
model.add(keras.layers.Dense(10, activation="relu"))

'''



In [69]:
print(len(X))
print(len(y))

27054
27054


In [77]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=64,kernel_size = 2, activation='relu', input_shape=(10,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='relu'))



In [ ]:
'''
hu1 = 10
hu2 = 6
afunc = tf.nn.relu
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(hu1, activation=afunc))
'''

In [ ]:
'''
hu1 = 10
hu2 = 6
afunc = tf.nn.relu

model = keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 9, activation = 'relu', input_dim = 16))
model.add(tf.keras.layers.Dense(hu1, activation=afunc))
model.add(tf.keras.layers.Dense(hu2, activation=afunc))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
'''

In [78]:
optim = 'adam'
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [30]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 21, 86)            1892      
_________________________________________________________________
dense_8 (Dense)              (None, 21, 50)            4350      
_________________________________________________________________
dense_9 (Dense)              (None, 21, 1)             51        
Total params: 6,293
Trainable params: 6,293
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.fit(X_train, y_train, epochs=500)

ValueError: Error when checking input: expected conv1d_7_input to have shape (10, 1) but got array with shape (21, 1)

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(val_loss)
print(val_acc)

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss)
print(val_acc)